In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp

In [ ]:
pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'

In [ ]:
d = tables_io.read(f"{pz_dir}/data/on_sky/dp1_v29.0.0_gold_ECDFS.hdf5")

In [ ]:
flux_type = 'psf'
bands = 'ugrizy'

In [ ]:
def pretty_plot(d, flux_type, bands):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        axs.hist(d[f"{band}_{flux_type}Mag"], np.linspace(15, 30, 151))
        axs.set_xlabel(f"{band} [mag]")
    fig.tight_layout()

In [ ]:
pretty_plot(d, flux_type, bands)